In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt

# get the region of interest
def sel_extent(ds):
    return ds.sel(lat=slice(6,36),lon=slice(68,98))

In [2]:
dd = {}
dd["pm"] = sel_extent(xr.open_dataset("../data/daily_surface_pm25_RH50.nc"))
dd["emission"] = sel_extent(xr.open_dataset("../data/daily_emission.nc"))
dd["gas"] = sel_extent(xr.open_dataset("../data/daily_gas_column.nc"))
dd["aod"] = sel_extent(xr.open_dataset("../data/daily_aod.nc"))
dd["met"] = sel_extent(xr.open_dataset("../data/daily_meteo.nc"))

mask = xr.open_dataset("../data/land_mask.nc")
r_mask = xr.open_dataset("../data/r_mask.nc",engine="scipy")

In [3]:
# sum over the dust, BC, and OC
dd["emission"]["EmisDST_Natural"] = dd["emission"]["EmisDST1_Natural"]\
                                  + dd["emission"]["EmisDST2_Natural"]\
                                  + dd["emission"]["EmisDST3_Natural"]\
                                  + dd["emission"]["EmisDST4_Natural"]
dd["emission"]["EmisBC_Anthro"] = dd["emission"]["EmisBCPI_Anthro"]\
                                + dd["emission"]["EmisBCPO_Anthro"]
dd["emission"]["EmisBC_BioBurn"] = dd["emission"]["EmisBCPI_BioBurn"]\
                                 + dd["emission"]["EmisBCPO_BioBurn"]
dd["emission"]["EmisOC_Anthro"] = dd["emission"]["EmisOCPI_Anthro"]\
                                + dd["emission"]["EmisOCPO_Anthro"]
dd["emission"]["EmisOC_BioBurn"] = dd["emission"]["EmisOCPI_BioBurn"]\
                                 + dd["emission"]["EmisOCPO_BioBurn"]

In [4]:
feature_ml = {
    "pm":["PM25"],
    "gas":['CO_trop', 'SO2_trop', 'NO2_trop', 'CH2O_trop', 'NH3_trop'],
    "aod":['AOT_C', 'AOT_DUST_C'],
    "met":['T2M', 'PBLH', 'U10M', 'V10M', 'PRECTOT', 'RH'],
    "emission":['EmisDST_Natural', 
                'EmisNO_Fert', 'EmisNO_Lightning', 'EmisNO_Ship', 'EmisNO_Soil',
                'EmisBC_Anthro', 'EmisBC_BioBurn', 
                'EmisCH2O_Anthro', 'EmisCH2O_BioBurn', 
                'EmisCO_Anthro', 'EmisCO_BioBurn', 'EmisCO_Ship', 
                'EmisNH3_Anthro', 'EmisNH3_BioBurn', 'EmisNH3_Natural', 
                'EmisNO_Aircraft', 'EmisNO_Anthro', 'EmisNO_BioBurn', 
                'EmisOC_Anthro', 'EmisOC_BioBurn',  
                'EmisSO2_Aircraft', 'EmisSO2_Anthro', 'EmisSO2_BioBurn',
                'EmisSO4_Anthro']
}

feature_ls = [feature_ml[k] for k in feature_ml] #if k!="pm"]
# ref https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-a-list-of-lists
feature_ls = sum(feature_ls,[])

## with mask

In [5]:
ds = xr.merge(dd[k][feature_ml[k]] for k in dd)\
       .groupby('time.month').mean(dim=["time"]).rename({"month":"time"}).where(mask["mask"])
loc_name = list(r_mask)
df_train_ls = []
df_test_ls = []

for loc in loc_name:
    # df_train
    train_tmp = ds.sel(time=ds.time.isin([1,2,3,5,6,7,9,10,11,12]))\
                  .where(r_mask[loc]).to_dataframe().reset_index().dropna()
    train_tmp["region"] = loc
    df_train_ls.append(train_tmp.copy())
    # df_test
    test_tmp = ds.sel(time=ds.time.isin([4,8]))\
                  .where(r_mask[loc]).to_dataframe().reset_index().dropna()
    test_tmp["region"] = loc
    df_test_ls.append(test_tmp.copy())
    
train = pd.concat(df_train_ls)
test = pd.concat(df_test_ls)
del df_train_ls, df_test_ls
gc.collect()
print("shape of train:", train.shape)
print("shape of test:", test.shape)

shape of train: (39230, 42)
shape of test: (7846, 42)


## save and load

In [6]:
# save
train.to_parquet('../data/c_r_monthly_train.gzip', compression='gzip') 
test.to_parquet('../data/c_r_monthly_test.gzip', compression='gzip') 

print("shape of train:", train.shape)
print("shape of test:", test.shape)

# load 
train_l = pd.read_parquet('../data/c_r_monthly_train.gzip')
test_l = pd.read_parquet('../data/c_r_monthly_test.gzip')

print("shape of train_l:", train_l.shape)
print("shape of test_l:", test_l.shape)

# test
np.testing.assert_array_equal(train[feature_ls].values, train_l[feature_ls].values)
np.testing.assert_array_equal(test[feature_ls].values, test_l[feature_ls].values)
print("passed the test!")

shape of train: (39230, 42)
shape of test: (7846, 42)
shape of train_l: (39230, 42)
shape of test_l: (7846, 42)
passed the test!
